In [8]:
# Import findspark and initialize.
import findspark
findspark.init()

In [9]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [10]:
# 1. Read the home_sales_revised.csv from the provided AWS S3 bucket location into a PySpark DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"


In [13]:
import os

# Download the file using wget
!wget https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv

# Now read it into a Spark DataFrame
df = spark.read.csv("home_sales_revised.csv", header=True, inferSchema=True)

# Create the temporary view
df.createOrReplaceTempView("home_sales")

# Verify it's loaded
df.show(5)


--2025-04-17 20:29:20--  https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv
Resolving 2u-data-curriculum-team.s3.amazonaws.com (2u-data-curriculum-team.s3.amazonaws.com)... 3.5.13.54, 54.231.139.33, 52.217.123.33, ...
Connecting to 2u-data-curriculum-team.s3.amazonaws.com (2u-data-curriculum-team.s3.amazonaws.com)|3.5.13.54|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2747874 (2.6M) [text/csv]
Saving to: ‘home_sales_revised.csv’

home_sales_revised. 100%[===================>]   2.62M  14.6MB/s    in 0.2s    

2025-04-17 20:29:20 (14.6 MB/s) - ‘home_sales_revised.csv’ saved [2747874/2747874]

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+----

In [15]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
query = """
SELECT
    YEAR(date) AS year,
    ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms = 4
GROUP BY YEAR(date)
ORDER BY year
"""

avg_price_df = spark.sql(query)
avg_price_df.show()


+----+---------+
|year|avg_price|
+----+---------+
|2019| 300263.7|
|2020|298353.78|
|2021|301819.44|
|2022|296363.88|
+----+---------+



In [17]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
query = """
SELECT
    date_built,
    ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3
GROUP BY date_built
ORDER BY date_built
"""

avg_price_by_year_built = spark.sql(query)
avg_price_by_year_built.show()



+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2010|292859.62|
|      2011|291117.47|
|      2012|293683.19|
|      2013|295962.27|
|      2014|290852.27|
|      2015| 288770.3|
|      2016|290555.07|
|      2017|292676.79|
+----------+---------+



In [22]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
query = """
SELECT
    date_built AS year_built,
    ROUND(AVG(price), 2) AS avg_price
FROM home_sales
WHERE
    bedrooms = 3 AND
    bathrooms = 3 AND
    floors = 2 AND
    sqft_living >= 2000
GROUP BY date_built
ORDER BY date_built
"""

avg_price_filtered = spark.sql(query)
avg_price_filtered.show()




+----------+---------+
|year_built|avg_price|
+----------+---------+
|      2010|285010.22|
|      2011|276553.81|
|      2012|307539.97|
|      2013|303676.79|
|      2014|298264.72|
|      2015|297609.97|
|      2016| 293965.1|
|      2017|280317.58|
+----------+---------+



In [24]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.
import time

# Start timer
start_time = time.time()

# SQL query to get average price per view rating, with the specified conditions
query = """
SELECT
    view,
    ROUND(AVG(price), 2) AS avg_price
FROM home_sales
GROUP BY view
HAVING avg_price >= 350000
ORDER BY view DESC
"""

# Execute query
avg_price_per_view = spark.sql(query)

# Show results
avg_price_per_view.show()

# End timer and print the run time
end_time = time.time()
execution_time = end_time - start_time
print(f"Query executed in {execution_time:.4f} seconds")


+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

Query executed in 1.7111 seconds


In [25]:
# 7. Cache the the temporary table home_sales.
spark.catalog.cacheTable("home_sales")

# Verify the cache status
print("Is home_sales cached? ", spark.catalog.isCached("home_sales"))


Is home_sales cached?  True


In [26]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [27]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.
# Start timer for cached query
start_time_cached = time.time()

# Run the same SQL query using the cached data
query_cached = """
SELECT
    view,
    ROUND(AVG(price), 2) AS avg_price
FROM home_sales
GROUP BY view
HAVING avg_price >= 350000
ORDER BY view DESC
"""

# Execute the cached query
avg_price_per_view_cached = spark.sql(query_cached)

# Show results
avg_price_per_view_cached.show()

# End timer for cached query and calculate the runtime
end_time_cached = time.time()
execution_time_cached = end_time_cached - start_time_cached
print(f"Cached query executed in {execution_time_cached:.4f} seconds")


+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

Cached query executed in 1.5997 seconds


In [28]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
# Save the home_sales DataFrame as Parquet, partitioned by the "date_built" field
df.write.partitionBy("date_built").parquet("/path/to/save/home_sales_parquet")

print("Data partitioned and saved as Parquet.")


Data partitioned and saved as Parquet.


In [29]:
# 11. Read the formatted parquet data.
# Path to the directory where the Parquet files were saved
parquet_path = "/path/to/save/home_sales_parquet"

# Read the partitioned Parquet data
df_parquet = spark.read.parquet(parquet_path)

# Show the first few rows of the DataFrame
df_parquet.show(5)


+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|2ed8d509-7372-46d...|2021-08-06|258710|       3|        3|       1918|    9666|     1|         0|  25|      2015|
|941bad30-eb49-4a7...|2020-05-09|229896|       3|        3|       2197|    8641|     1|         0|   3|      2015|
|c797ca12-52cd-4b1...|2019-06-08|288650|       2|        3|       2100|   10419|     2|         0|   7|      2015|
|0cfe57f3-28c2-472...|2019-10-04|308313|       3|        3|       1960|    9453|     2|         0|   2|      2015|
|d715f295-2fbf-4e9...|2021-05-17|391574|       3|        2|       1635|    8040|     2|         0|  10|      2015|
+--------------------+----------+------+--------+---------+-----------+--------+

In [30]:
# 12. Create a temporary table for the parquet data.
df_parquet.createOrReplaceTempView("home_sales_parquet")

# Verify the temporary table by running a simple SQL query
spark.sql("SELECT * FROM home_sales_parquet LIMIT 5").show()


+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|2ed8d509-7372-46d...|2021-08-06|258710|       3|        3|       1918|    9666|     1|         0|  25|      2015|
|941bad30-eb49-4a7...|2020-05-09|229896|       3|        3|       2197|    8641|     1|         0|   3|      2015|
|c797ca12-52cd-4b1...|2019-06-08|288650|       2|        3|       2100|   10419|     2|         0|   7|      2015|
|0cfe57f3-28c2-472...|2019-10-04|308313|       3|        3|       1960|    9453|     2|         0|   2|      2015|
|d715f295-2fbf-4e9...|2021-05-17|391574|       3|        2|       1635|    8040|     2|         0|  10|      2015|
+--------------------+----------+------+--------+---------+-----------+--------+

In [31]:
# Start timer for query on Parquet DataFrame
start_time_parquet = time.time()

# Run the SQL query using the parquet data
query_parquet = """
SELECT
    view,
    ROUND(AVG(price), 2) AS avg_price
FROM home_sales_parquet
GROUP BY view
HAVING avg_price >= 350000
ORDER BY view DESC
"""

# Execute the query on the parquet data
avg_price_per_view_parquet = spark.sql(query_parquet)

# Show results
avg_price_per_view_parquet.show()

# End timer for parquet query and calculate the runtime
end_time_parquet = time.time()
execution_time_parquet = end_time_parquet - start_time_parquet
print(f"Query on Parquet data executed in {execution_time_parquet:.4f} seconds")


+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

Query on Parquet data executed in 0.9106 seconds


In [34]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("home_sales")


In [35]:
# 15. Check if the home_sales is no longer cached
is_cached = spark.catalog.isCached("home_sales")
print(f"Is home_sales cached? {is_cached}")

Is home_sales cached? False
